### PydanticOutputParser

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import os

llm = ChatOpenAI(temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL"))

In [2]:
email_conversation = """From: 교육담당자 (edu@sw.or.kr)"
To: 노규남 (bardroh@weable.ai)
Subject: 4월 회원지원 교육 안내

 문의사항이 있으신 경우 담당자에게 연락 바랍니다.

* 선착순 모집이며 조기 마감될 수 있습니다.

* 오프라인과 온라인 수업이 동시에 있는 경우, 동시 접수가 불가능합니다.

* 4월 24일(목) 종합소득세와 소득세 원천징수 온라인 교육이 있습니다.

그외의 교육은 오프라인 교육입니다.

* 온라인으로 교육 참여하시는 경우 교재 파일 링크를 제공해드리며 별도 실물 교재는 제공해 드리지 않습니다.
"""

In [3]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain = prompt | llm
chain.invoke({"email_conversation": email_conversation})

AIMessage(content='중요한 내용 요약:\n\n- **교육 일정**: 4월 24일(목) 종합소득세와 소득세 원천징수 온라인 교육\n- **모집 방식**: 선착순 모집, 조기 마감 가능\n- **수업 형태**: 오프라인과 온라인 수업 동시 접수 불가\n- **교재 제공**: 온라인 참여 시 교재 파일 링크 제공, 실물 교재는 제공되지 않음\n- **문의**: 문의사항은 담당자에게 연락 필요', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 173, 'total_tokens': 290, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEaMk8DNlgI05vcTUDD5X3uBaOJje', 'finish_reason': 'stop', 'logprobs': None}, id='run-c077f136-a321-4f25-a48a-6321c6c5f8dc-0', usage_metadata={'input_tokens': 173, 'output_tokens': 117, 'total_tokens': 290, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 날짜")

parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 날짜", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [5]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt = prompt.partial(format=parser.get_format_instructions())

In [6]:
chain = prompt | llm
answer = chain.invoke({"email_conversation": email_conversation,
              "question": "다음의 이메일 내용중 중요한 내용을 추출해 주세요."
})
print(answer)

content='```json\n{\n  "person": "교육담당자",\n  "email": "edu@sw.or.kr",\n  "subject": "4월 회원지원 교육 안내",\n  "summary": "4월 24일(목) 종합소득세와 소득세 원천징수 온라인 교육이 있으며, 선착순 모집으로 조기 마감될 수 있습니다. 온라인 교육 참여 시 실물 교재는 제공되지 않고, 교재 파일 링크가 제공됩니다.",\n  "date": "2024-04-24"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 474, 'total_tokens': 591, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEaRuEfMznu9yYrcX0eTSOtYqYtes', 'finish_reason': 'stop', 'logprobs': None} id='run-4f3f3189-cf5c-4d91-a568-9b9854a6ad93-0' usage_metadata={'input_tokens': 474, 'output_tokens': 117, 'total_tokens': 591, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reaso

In [7]:
structured_output = parser.parse(answer.content)
print(structured_output)

person='교육담당자' email='edu@sw.or.kr' subject='4월 회원지원 교육 안내' summary='4월 24일(목) 종합소득세와 소득세 원천징수 온라인 교육이 있으며, 선착순 모집으로 조기 마감될 수 있습니다. 온라인 교육 참여 시 실물 교재는 제공되지 않고, 교재 파일 링크가 제공됩니다.' date='2024-04-24'


In [8]:
chain = prompt | llm | parser
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)
response

EmailSummary(person='교육담당자', email='edu@sw.or.kr', subject='4월 회원지원 교육 안내', summary='4월 24일(목) 종합소득세와 소득세 원천징수 온라인 교육이 있으며, 선착순 모집으로 조기 마감될 수 있습니다. 오프라인 교육도 진행되며, 온라인 참여 시 교재 파일 링크만 제공됩니다.', date='2024-04-24')

In [9]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL")
).with_structured_output(EmailSummary)

answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='노규남', email='edu@sw.or.kr', subject='4월 회원지원 교육 안내', summary='4월 24일(목)에 종합소득세와 소득세 원천징수 온라인 교육이 있으며, 선착순 모집으로 조기 마감될 수 있습니다. 오프라인 교육도 진행되며, 온라인 참여 시 교재 파일 링크가 제공됩니다. 동시 접수는 불가능합니다.', date='2023-04-24')

### CommaSeparatedListOutputParser

In [10]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [11]:

prompt = PromptTemplate(
    template="List 10 {subject}.\n{format_instructions}",
    input_variables=["subject"], 
    partial_variables={"format_instructions": format_instructions}, # 미리 값이 지정되어 있으나 오버라이드 가능한 변수
)

model = ChatOpenAI(temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL"))

chain = prompt | model | output_parser

In [12]:
chain.invoke({"subject": "미국 국립공원"})

['옐로스톤',
 '그랜드 캐니언',
 '요세미티',
 '자이언',
 '스모키 마운틴',
 '아카디아',
 '시에라 네바다',
 '올림픽',
 '그레이트 스모키 마운틴',
 '세쿼이아']

### StructuredOuputParser

In [13]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 출처의 웹사이트 주소를 기록합니다.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [14]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // 사용자의 질문에 대한 답변\n\t"source": string  // 사용자의 질문에 답하기 위해 사용된 출처의 웹사이트 주소를 기록합니다.\n}\n```'

In [15]:
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [16]:
model = ChatOpenAI(temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL"))
chain = prompt | model | output_parser  

In [17]:
chain.invoke({"question": "OpenAI의 창업자는 누구인가요?"})

{'answer': 'OpenAI의 창립자는 엘론 머스크(Elon Musk), 샘 알트만(Sam Altman), 그렉 브록만(Greg Brockman), Ilya Sutskever, Wojciech Zaremba, John Schulman 등입니다. 이들은 2015년에 OpenAI를 설립했습니다.',
 'source': 'https://openai.com/about/'}

### JsonOutputParser

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [19]:
model = ChatOpenAI(temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL"))

In [20]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    keywords: str = Field(description="설명에 대한 주요 키워드(2개 이상)")

In [21]:
question = "도널드 트럼프의 외교정책에 대해서 설명해주세요."

parser = JsonOutputParser(pydantic_object=Topic)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간단하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  

chain.invoke({"question": question})

{'description': "도널드 트럼프의 외교정책은 '미국 우선주의'를 중심으로 하여 무역 협정 재협상, NATO 동맹국의 방위비 분담 증대 요구, 이란 핵 합의 탈퇴, 중국과의 무역 전쟁 등을 포함합니다.",
 'keywords': '미국 우선주의, 무역 전쟁'}

In [22]:
question = "도널드 트럼프의 외교정책에 대해서 설명해주세요."
parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간단하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"question": question})

print(response)

{'도널드_트럼프의_외교정책': {'주요_특징': ['미국 우선주의: 미국의 이익을 최우선으로 고려하는 정책.', '무역전쟁: 중국과의 무역 갈등을 통해 관세를 부과하고 무역 불균형 해소를 목표로 함.', '북한과의 대화: 김정은과의 정상회담을 통해 핵 문제 해결을 시도.', 'NATO 및 동맹국에 대한 압박: 동맹국들이 방위비를 더 많이 부담하도록 요구.', '이란 핵 합의 탈퇴: 이란과의 핵 합의를 폐기하고 제재를 강화.'], '비판': ['국제 협력 약화: 다자간 협력보다는 일방적인 접근을 선호.', '동맹국과의 관계 악화: 전통적인 동맹국들과의 신뢰 관계가 약화됨.', '무역 갈등의 부작용: 글로벌 경제에 부정적인 영향을 미침.']}}


In [23]:
question = "도널드 트럼프의 외교정책에 대해서 설명해주세요. 설명은 'description'에, 키워드는 'keywords'에 기록해주세요."

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간단하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"question": question})

print(response)

{'description': "도널드 트럼프의 외교정책은 '미국 우선주의'를 중심으로 하며, 무역 협정 재협상, NATO 및 동맹국에 대한 방위비 분담 요구, 이란 핵 합의 탈퇴, 북한과의 정상 회담 등으로 특징지어집니다. 또한, 중국과의 무역 전쟁을 통해 경제적 압박을 가하고, 이민 정책을 강화하는 등의 접근을 취했습니다.", 'keywords': ['미국 우선주의', '무역 협정', 'NATO', '방위비 분담', '이란 핵 합의', '북한 정상 회담', '중국 무역 전쟁', '이민 정책']}


### DatetimeOutputParser

In [24]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }
)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 1923-03-31, 1749-11-21, 201-01-20\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [25]:
chain = prompt | ChatOpenAI() | output_parser
output = chain.invoke({"question": "오바마 대통령이 취임한 날짜는 언제인가?"})
output

datetime.datetime(2009, 1, 20, 0, 0)

In [26]:
output.strftime("%Y-%m-%d")

'2009-01-20'

### EnumOutputParser

In [27]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

parser = EnumOutputParser(enum=Colors)
parser.get_format_instructions()

'Select one of the following options: 빨간색, 초록색, 파란색'

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI() | parser

In [29]:
response = chain.invoke({"object": "하늘"}) 
print(response)

Colors.BLUE


### OutputFixingParser

In [30]:
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

In [31]:
class Movie(BaseModel):
    title: str = Field(description="영화 제목")
    director: str = Field(description="영화 감독")
    year: int = Field(description="개봉 연도")
    genres: List[str] = Field(description="영화 장르 목록")

In [32]:
parser = PydanticOutputParser(pydantic_object=Movie)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "영화 제목", "title": "Title", "type": "string"}, "director": {"description": "영화 감독", "title": "Director", "type": "string"}, "year": {"description": "개봉 연도", "title": "Year", "type": "integer"}, "genres": {"description": "영화 장르 목록", "items": {"type": "string"}, "title": "Genres", "type": "array"}}, "required": ["title", "director", "year", "genres"]}\n```'

In [33]:
import os

llm = ChatOpenAI(temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL"))
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
fixing_parser.get_format_instructions()

/tmp/ipykernel_68858/304468780.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL"))


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "영화 제목", "title": "Title", "type": "string"}, "director": {"description": "영화 감독", "title": "Director", "type": "string"}, "year": {"description": "개봉 연도", "title": "Year", "type": "integer"}, "genres": {"description": "영화 장르 목록", "items": {"type": "string"}, "title": "Genres", "type": "array"}}, "required": ["title", "director", "year", "genres"]}\n```'

In [34]:
# 연도가 문자열이며 장르가 배열이 아님
incorrect_output = """
{
  "title": "기생충",
  "director": "봉준호",
  "year": "2019", 
  "genres": "드라마, 스릴러, 블랙 코미디"
}
"""

In [35]:
try:
    regular_result = parser.parse(incorrect_output) 
    print(regular_result)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Failed to parse Movie from completion {"title": "\uae30\uc0dd\ucda9", "director": "\ubd09\uc900\ud638", "year": "2019", "genres": "\ub4dc\ub77c\ub9c8, \uc2a4\ub9b4\ub7ec, \ube14\ub799 \ucf54\ubbf8\ub514"}. Got: 1 validation error for Movie
genres
  Input should be a valid list [type=list_type, input_value='드라마, 스릴러, 블랙 코미디', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [36]:
try:
    fixed_result = fixing_parser.parse(incorrect_output)
    print(fixed_result)
except Exception as e:
    print(f"오류 발생: {e}")

title='기생충' director='봉준호' year=2019 genres=['드라마', '스릴러', '블랙 코미디']


### Custom Output Parser

In [37]:
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [38]:
class CommaSeparatedListOutputParser(BaseOutputParser):
    """
    쉼표로 구분된 항목 목록을 파싱하는 간단한 출력 파서
    """
    def parse(self, text: str) -> list:
        cleaned_text = text.strip()
        if not cleaned_text:
            return []
        items = [item.strip() for item in cleaned_text.split(",")]
        return items

In [39]:
template = """
다음 주제에 관련된 항목을 5개 나열해주세요: {topic}

항목은 쉼표로 구분해 주세요.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["topic"],
)

output_parser = CommaSeparatedListOutputParser()

In [40]:
import os

llm = ChatOpenAI(temperature=0, model_name=os.getenv("OPENAI_DEFAULT_MODEL"))
chain = prompt | llm | output_parser

In [41]:
result = chain.invoke({"topic": "인공지능의 응용 분야"})
print(result) 
print(type(result)) 

['자연어 처리', '이미지 인식', '자율주행차', '의료 진단', '추천 시스템']
<class 'list'>


In [42]:
result2 = chain.invoke({"topic": "한국의 대표 음식"})
print(result2)

['김치', '비빔밥', '불고기', '떡볶이', '삼겹살']
